In [ ]:
import pandas as pd

weather_stations = pd.read_csv('/Users/evelyn/Desktop/project/weather-stations.csv', encoding= 'unicode_escape')

salmdata = pd.read_excel('/Users/evelyn/Desktop/Cleaned data - Salmonella and norovirus.xlsx', sheet_name = 'Salmonella')
norodata = pd.read_excel('/Users/evelyn/Desktop/Cleaned data - Salmonella and norovirus.xlsx', sheet_name = 'Norovirus')
                                     

In [3]:
pip install progressbar2

Note: you may need to restart the kernel to use updated packages.


# Find the corresponding weather stations to each county

In [141]:
import re
import os #Import for setting working dirrectory
import requests #Import for downloading data from database
from tqdm import tqdm
import sys
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Base URL and filename
url_base = "https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/" #Daily data path 
station = pd.DataFrame()
for i in range(len(salmdata)):
    print('{:.4%}'.format(i/len(salmdata)))
    df = weather_stations[weather_stations['STATE'].isin([salmdata.loc[i,"Exposurestate"]])]
    for j in range(0,19):
        if(pd.isna(salmdata.loc[i,"ExposureCounty"+str(j+1)])):
            break
        else:
            salm_station = pd.DataFrame(salmdata.iloc[i,:7]).T
            salm_station['County'] = salmdata.loc[i,"ExposureCounty"+str(j+1)]
            new_stations = df[df.County.str.contains(str(salmdata.loc[i,"ExposureCounty"+str(j+1)]),regex= True, na=False)]
            if(new_stations.empty):
                station = station.append(salm_station)
                station = station.reset_index(drop=True)
                continue
            else:
                #检查CSV file 完整性
                new_stations = new_stations.reset_index(drop=True)
                for z in range(len(new_stations)):
                    wban_unmodified = new_stations.loc[z, "WBAN"]
                    if 0 < wban_unmodified < 10000: 
                        new_stations.loc[z, "WBAN"] = str(wban_unmodified).zfill(5)
                    usaf = new_stations.loc[z, "USAF"]
                    #print(usaf) 
                    wban = new_stations.loc[z, "WBAN"]
                    #print(wban)
                    startyear = str(salm_station.loc[i , "InitialExposure"])[:4] #Extracts the year from the date string by taking the first four elements
                    #print(startyear)
                    endyear = str(salm_station.loc[i , "LastExposure"])[:4]  
                    #print(endyear)
                    filename_noaa = str(usaf) + str(wban) + ".csv" 
                    for yr in range(int(startyear), int(endyear)+1): 
                        url_loop_daily = url_base + str(yr) + "/" + filename_noaa
                        #print(url_loop_daily)

                        #Identifies file on website
                        csv_current_daily = requests.get(url_loop_daily, allow_redirects = True)

                        #Sets the file name of the local imported file
                        filename_local_daily =  str(usaf) + str(wban) + "-" + str(yr) + "DAILY.csv" 
                        os.chdir('/Users/evelyn/Desktop/salm') #Sets path for daily data csv deposit  
                        open(filename_local_daily, "wb").write(csv_current_daily.content) #Write a csv for daily data
                        total = sum(1 for line in open(filename_local_daily))
                        if(total < 35):
                            new_stations.loc[z, "Incom"] = 'No'
                        else:
                            new_stations.loc[z, "Incom"] = 'Yes'
                            
                new_stations = new_stations[~(new_stations["Incom"] == 'No')]
                new_stations = new_stations.dropna(subset = ['USAF'])
                new_stations = new_stations.reset_index(drop = True)
                if(new_stations.empty):
                    station = station.append(salm_station)
                    station = station.reset_index(drop=True)
                else:
                    salm_newstation = salm_station.append([salm_station]*(len(new_stations)-1))
                    salm_newstation = salm_newstation.reset_index(drop = True)
                    for index in range(len(new_stations)):
                        salm_newstation.loc[index, "USAF"] = new_stations.loc[index, "USAF"]
                        salm_newstation.loc[index, "WBAN"] = new_stations.loc[index, "WBAN"]
                    station = station.append(salm_newstation)
                    station = station.reset_index(drop = True)
                        
                                               

0.0000%
0.1063%
0.2125%
0.3188%
0.4251%
0.5313%
0.6376%
0.7439%
0.8502%
0.9564%
1.0627%
1.1690%
1.2752%
1.3815%
1.4878%
1.5940%
1.7003%
1.8066%
1.9129%
2.0191%
2.1254%
2.2317%
2.3379%
2.4442%
2.5505%
2.6567%
2.7630%
2.8693%
2.9756%
3.0818%
3.1881%
3.2944%
3.4006%
3.5069%
3.6132%
3.7194%
3.8257%
3.9320%
4.0383%
4.1445%
4.2508%
4.3571%
4.4633%
4.5696%
4.6759%
4.7821%
4.8884%
4.9947%
5.1010%
5.2072%
5.3135%
5.4198%
5.5260%
5.6323%
5.7386%
5.8448%
5.9511%
6.0574%
6.1637%
6.2699%
6.3762%
6.4825%
6.5887%
6.6950%
6.8013%
6.9075%
7.0138%
7.1201%
7.2264%
7.3326%
7.4389%
7.5452%
7.6514%
7.7577%
7.8640%
7.9702%
8.0765%
8.1828%
8.2891%
8.3953%
8.5016%
8.6079%
8.7141%
8.8204%
8.9267%
9.0329%
9.1392%
9.2455%
9.3518%
9.4580%
9.5643%
9.6706%
9.7768%
9.8831%
9.9894%
10.0956%
10.2019%
10.3082%
10.4145%
10.5207%
10.6270%
10.7333%
10.8395%
10.9458%
11.0521%
11.1583%
11.2646%
11.3709%
11.4772%
11.5834%
11.6897%
11.7960%
11.9022%
12.0085%
12.1148%
12.2210%
12.3273%
12.4336%
12.5399%
12.6461%
12.7524%
12.858

In [142]:
station.isnull().sum()

CDCID                0
GenusName            0
InitialExposure      0
LastExposure         0
TotalCases           0
Exposurestate        1
ExposureState        0
County               0
USAF               209
WBAN               209
dtype: int64

# Download the CSV files of matched counties

In [143]:
url_base = "https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/" #Daily data path 
station["WBAN"] = pd.to_numeric(station["WBAN"], errors='coerce').fillna('0').astype('int32')
for z in range(len(station)):
    wban_unmodified = station.loc[z, "WBAN"]
    if 0 < wban_unmodified < 10000:
        
        station.loc[z, "WBAN"] = str(wban_unmodified).zfill(5)
for i in range(len(station)):
    print('{:.4%}'.format(i/len(station)))
    if(pd.isna(station.loc[i, "USAF"])):
        continue
    else:
        usaf = station.loc[i, "USAF"]
                    #print(usaf) 
        wban = station.loc[i, "WBAN"]
                    #print(wban)
        startyear = str(station.loc[i , "InitialExposure"])[:4] #Extracts the year from the date string by taking the first four elements
                    #print(startyear)
        endyear = str(station.loc[i , "LastExposure"])[:4]  
                    #print(endyear)
        filename_noaa = str(usaf) + str(wban) + ".csv" 
        for yr in range(int(startyear), int(endyear)+1): 
            
            url_loop_daily = url_base + str(yr) + "/" + filename_noaa
                        #print(url_loop_daily)

                        #Identifies file on website
            csv_current_daily = requests.get(url_loop_daily, allow_redirects = True)

                        #Sets the file name of the local imported file
            filename_local_daily =  str(usaf) + str(wban) + "-" + str(yr) + "DAILY.csv" 
            os.chdir('/Users/evelyn/Desktop/salm') #Sets path for daily data csv deposit  
            open(filename_local_daily, "wb").write(csv_current_daily.content) #Write a csv for daily data
        
        
    
    

0.0000%
0.0301%
0.0603%
0.0904%
0.1206%
0.1507%
0.1808%
0.2110%
0.2411%
0.2712%
0.3014%
0.3315%
0.3617%
0.3918%
0.4219%
0.4521%
0.4822%
0.5124%
0.5425%
0.5726%
0.6028%
0.6329%
0.6631%
0.6932%
0.7233%
0.7535%
0.7836%
0.8137%
0.8439%
0.8740%
0.9042%
0.9343%
0.9644%
0.9946%
1.0247%
1.0549%
1.0850%
1.1151%
1.1453%
1.1754%
1.2055%
1.2357%
1.2658%
1.2960%
1.3261%
1.3562%
1.3864%
1.4165%
1.4467%
1.4768%
1.5069%
1.5371%
1.5672%
1.5973%
1.6275%
1.6576%
1.6878%
1.7179%
1.7480%
1.7782%
1.8083%
1.8385%
1.8686%
1.8987%
1.9289%
1.9590%
1.9892%
2.0193%
2.0494%
2.0796%
2.1097%
2.1398%
2.1700%
2.2001%
2.2303%
2.2604%
2.2905%
2.3207%
2.3508%
2.3810%
2.4111%
2.4412%
2.4714%
2.5015%
2.5316%
2.5618%
2.5919%
2.6221%
2.6522%
2.6823%
2.7125%
2.7426%
2.7728%
2.8029%
2.8330%
2.8632%
2.8933%
2.9234%
2.9536%
2.9837%
3.0139%
3.0440%
3.0741%
3.1043%
3.1344%
3.1646%
3.1947%
3.2248%
3.2550%
3.2851%
3.3153%
3.3454%
3.3755%
3.4057%
3.4358%
3.4659%
3.4961%
3.5262%
3.5564%
3.5865%
3.6166%
3.6468%
3.6769%
3.7071%
3.7372%


In [368]:
station.to_csv('/Users/evelyn/Desktop/new-/salm_station.csv')

In [30]:
import pandas as pd
station = pd.read_csv('/Users/evelyn/Desktop/new-/salm_station.csv', dtype = 'str')

In [31]:
station["WBAN"] = pd.to_numeric(station["WBAN"], errors='coerce').fillna('0').astype('int32')
for a in range(len(station)):
    wban_unmodified = station.loc[a, "WBAN"]
    if 0 < wban_unmodified < 10000: 
        station.loc[a, "WBAN"] = str(wban_unmodified).zfill(5)

In [32]:
# tackle the SNDP and FRSHTT information in matched counties
path = '/Users/evelyn/Desktop/salm/'
store_path = '/Users/evelyn/Desktop/new-/salm-weather/'
for i in range(len(station)):
    print('{:.4%}'.format(i/len(station)))
    if(pd.isna(station.loc[i, "USAF"])):
        continue
    else:
        usaf = station.loc[i, "USAF"]
                    #print(usaf) 
        wban = station.loc[i, "WBAN"]
        startyear = str(station.loc[i , "InitialExposure"])[:4] #Extracts the year from the date string by taking the first four elements
                    #print(startyear)
        endyear = str(station.loc[i , "LastExposure"])[:4]  
        for yr in range(int(startyear), int(endyear)+1): 
            filename = str(usaf) + str(wban) + "-" + str(yr) + "DAILY.csv" 
            df = pd.read_csv(path + filename)
            for j in range(len(df)):
                if not 'SNDP' in df.columns:
                    break
                else:
                    if(df.loc[j, "SNDP"] > 999):
                        df.loc[j, "SNDP"] = 0
                    if((df.loc[j, "FRSHTT"]%10) > 0):# 个位
                        df.loc[j, "Tornado or Funnel Cloud"] = int(1)
                    else:
                        df.loc[j, "Tornado or Funnel Cloud"] = int(0)
                    if((df.loc[j, "FRSHTT"]//10%10) > 0):#十位
                        df.loc[j, "Thunder"] = int(1)
                    else:
                        df.loc[j, "Thunder"] = int(0)
                    if((df.loc[j, "FRSHTT"]//100%10) > 0):#百位
                        df.loc[j, "Hail"] = int(1)
                    else:
                        df.loc[j, "Hail"] = int(0)
                    if((df.loc[j, "FRSHTT"]//1000%10) > 0):#千位
                        df.loc[j, "Snow or Ice Pellets"] = int(1)
                    else:
                        df.loc[j, "Snow or Ice Pellets"] = int(0)
                    if((df.loc[j, "FRSHTT"]//10000 % 10) > 0):
                        df.loc[j, "Rain or Drizzle"] = int(1)
                    else:
                        df.loc[j, "Rain or Drizzle"] = int(0)
                    if((df.loc[j, "FRSHTT"]//100000 % 10) > 0):
                        df.loc[j, "Fog"] = int(1)
                    else:
                        df.loc[j, "Fog"] = int(0)

            df.to_csv(store_path + filename)
            

0.0000%
0.0301%
0.0603%
0.0904%
0.1206%
0.1507%
0.1808%
0.2110%
0.2411%
0.2712%
0.3014%
0.3315%
0.3617%
0.3918%
0.4219%
0.4521%
0.4822%
0.5124%
0.5425%
0.5726%
0.6028%
0.6329%
0.6631%
0.6932%
0.7233%
0.7535%
0.7836%
0.8137%
0.8439%
0.8740%
0.9042%
0.9343%
0.9644%
0.9946%
1.0247%
1.0549%
1.0850%
1.1151%
1.1453%
1.1754%
1.2055%
1.2357%
1.2658%
1.2960%
1.3261%
1.3562%
1.3864%
1.4165%
1.4467%
1.4768%
1.5069%
1.5371%
1.5672%
1.5973%
1.6275%
1.6576%
1.6878%
1.7179%
1.7480%
1.7782%
1.8083%
1.8385%
1.8686%
1.8987%
1.9289%
1.9590%
1.9892%
2.0193%
2.0494%
2.0796%
2.1097%
2.1398%
2.1700%
2.2001%
2.2303%
2.2604%
2.2905%
2.3207%
2.3508%
2.3810%
2.4111%
2.4412%
2.4714%
2.5015%
2.5316%
2.5618%
2.5919%
2.6221%
2.6522%
2.6823%
2.7125%
2.7426%
2.7728%
2.8029%
2.8330%
2.8632%
2.8933%
2.9234%
2.9536%
2.9837%
3.0139%
3.0440%
3.0741%
3.1043%
3.1344%
3.1646%
3.1947%
3.2248%
3.2550%
3.2851%
3.3153%
3.3454%
3.3755%
3.4057%
3.4358%
3.4659%
3.4961%
3.5262%
3.5564%
3.5865%
3.6166%
3.6468%
3.6769%
3.7071%
3.7372%


In [7]:
from math import radians, cos, sin, asin, sqrt, atan
from math import *
def haversine(lat1, lon1, lat2, lon2):
    
    R = 3959.87433 #Earth radius in miles
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat/2)**2 + sin(dlon/2)**2 * cos(lat1) * cos(lat2)
    c = 2 * asin(sqrt(a))
    return R*c          

# unmatched counties weather information by using interpolation

In [6]:
import pandas as pd
salm_null = pd.read_csv('/Users/evelyn/Desktop/salm_null.csv')
weather_stations = pd.read_csv('/Users/evelyn/Desktop/project/weather-stations.csv', encoding= 'unicode_escape')

salm_null

,CDCID,GenusName,InitialExposure,LastExposure,TotalCases,Exposurestate,ExposureState,County,county,LAT,...,DIS0,USAF1,WBAN1,DIS1,USAF2,WBAN2,DIS2,W0,W1,W2
0,2118,Salmonella,2009/5/7,2009/5/20,5,MO,Missouri,Cass,Cass County,38.646474,...,21.548677,724467,13930.0,43.679216,999999,23908.0,85.125338,0.572580,0.282476,0.144943
1,2118,Salmonella,2009/5/7,2009/5/20,5,MO,Missouri,Christian,Christian County,36.969739,...,21.756157,723450,13971.0,48.597971,723439,53918.0,57.323484,0.547283,0.245005,0.207712
2,2118,Salmonella,2009/5/7,2009/5/20,5,MO,Missouri,Jefferson,Jefferson County,38.257630,...,34.896776,724340,13994.0,35.392365,724420,13997.0,66.903272,0.398788,0.393204,0.208008
3,2621,Salmonella,2009/9/26,2009/10/11,16,IL,Illinois,McHenry,McHenry County,42.324298,...,30.879758,725430,94822.0,34.008514,726415,94854.0,35.857541,0.361118,0.327895,0.310987
4,2627,Salmonella,2009/5/25,2009/6/11,21,WV,West Virginia,Logan,Logan County,37.830591,...,26.222460,720160,63884.0,26.222460,742079,63876.0,75.434824,0.425964,0.425964,0.148072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,298455,Salmonella,2021/8/3,2021/8/7,11,KY,Kentucky,Lyon,Lyon County,37.023976,...,36.456140,746710,13806.0,41.368347,722781,99999.0,50.233914,0.383585,0.338037,0.278378
206,298611,Salmonella,2021/9/25,2021/9/25,5,CT,Connecticut,Tolland,Tolland County,41.850000,...,10.670789,725087,14752.0,18.350952,725080,14740.0,19.081355,0.467135,0.271631,0.261234
207,302498,Salmonella,2021/3/21,2021/3/21,11,LA,Louisiana,Franklin,Franklin Parish,32.139076,...,33.389064,722355,93999.0,43.731498,722487,13935.0,62.648087,0.435452,0.332468,0.232080
208,16626,Salmonella,2013/9/6,2013/9/20,15,KY,Kentucky,Hopkins,NaN,37.311070,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
import heapq
import re
import os #Import for setting working dirrectory
import requests #Import for downloading data from database
from tqdm import tqdm
import sys
import numpy as np
url_base = "https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/"
weather_stations["WBAN"] = pd.to_numeric(weather_stations["WBAN"], errors='coerce').fillna('0').astype('int32')
for z in range(len(weather_stations)):
    wban_unmodified = weather_stations.loc[z, "WBAN"]
    if 0 < wban_unmodified < 10000: 
        
        weather_stations.loc[z, "WBAN"] = str(wban_unmodified).zfill(5)
        
i = salm_null[salm_null.CDCID == c[i]].index.tolist()[0]
slat = float(salm_null.loc[i, "LAT"])
slon = float(salm_null.loc[i, "LON"])
for j in range(len(weather_stations)):
    lat = float(weather_stations.loc[j, "LAT"])
    lon = float(weather_stations.loc[j, "LON"])
    weather_stations.loc[j, "DIS"] = haversine(slat, slon, lat, lon)
dis = weather_stations["DIS"].tolist()

weather_stations["WBAN"] = pd.to_numeric(weather_stations["WBAN"], errors='coerce').astype('int32')
for l in range(60):
    z = []
    min_index = list(map(dis.index, heapq.nsmallest(3, dis)))
    dlat = [weather_stations.loc[min_index[0], "LAT"], weather_stations.loc[min_index[1], "LAT"], weather_stations.loc[min_index[2],"LAT"]]
    dlon = [weather_stations.loc[min_index[0], "LON"], weather_stations.loc[min_index[1], "LON"], weather_stations.loc[min_index[2],"LON"]]
    if(min(dlat) < slat and slat < max(dlat) and min(dlon) < slon and slon < max(dlon)):
        for m in range(3):
            usaf = weather_stations.loc[min_index[m], "USAF"]
            #print(usaf) 
            wban = weather_stations.loc[min_index[m], "WBAN"]
            #print(wban)
            startyear = str(salm_null.loc[i , "InitialExposure"])[:4] #Extracts the year from the date string by taking the first four elements
            #print(startyear)
            endyear = str(salm_null.loc[i , "LastExposure"])[:4]  
            #print(endyear)
            filename_noaa = str(usaf) + str(wban) + ".csv" 
            for yr in range(int(startyear), int(endyear)+1): 
                url_loop_daily = url_base + str(yr) + "/" + filename_noaa
                #print(url_loop_daily)

                #Identifies file on website
                csv_current_daily = requests.get(url_loop_daily, allow_redirects = True)

                #Sets the file name of the local imported file
                filename_local_daily =  str(usaf) + str(wban) + "-" + str(yr) + "DAILY.csv" 
                os.chdir('/Users/evelyn/Desktop/project/salm_null') #Sets path for daily data csv deposit  
                open(filename_local_daily, "wb").write(csv_current_daily.content) #Write a csv for daily data
                total = sum(1 for line in open(filename_local_daily))
                if(total < 30):
                    dis[min_index[m]] = 100
                    z.append(100)
                else:
                    z.append(0)
        if(max(z) < 50):
            break
        else: 
            continue
    else:
        dis[min_index[2]] = 100

for k in range(3):
    index = min_index[k]
    salm_null.loc[i, "USAF"+str(k)] = weather_stations.loc[index, "USAF"]
    salm_null.loc[i, "WBAN"+str(k)] = weather_stations.loc[index, "WBAN"]
    salm_null.loc[i, "DIS"+str(k)] = dis[index]
    


In [17]:
salm_null.to_csv('/Users/evelyn/Desktop/salm_null.csv')

In [226]:
import heapq
url_base = "https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/"
weather_stations["WBAN"] = pd.to_numeric(weather_stations["WBAN"], errors='coerce').fillna('0').astype('int32')
for z in range(len(weather_stations)):
    wban_unmodified = weather_stations.loc[z, "WBAN"]
    if 0 < wban_unmodified < 10000: 
        
        weather_stations.loc[z, "WBAN"] = str(wban_unmodified).zfill(5)

for i in range(len(salm_null)):
    print('{:.4%}'.format(i/len(salm_null)))
    slat = float(salm_null.loc[i, "LAT"])
    slon = float(salm_null.loc[i, "LON"])
    for j in range(len(weather_stations)):
        lat = float(weather_stations.loc[j, "LAT"])
        lon = float(weather_stations.loc[j, "LON"])
        weather_stations.loc[j, "DIS"] = haversine(slat, slon, lat, lon)
    dis = weather_stations["DIS"].tolist()
    
    weather_stations["WBAN"] = pd.to_numeric(weather_stations["WBAN"], errors='coerce').astype('int32')
    for l in range(60):
        z = []
        min_index = list(map(dis.index, heapq.nsmallest(3, dis)))
        dlat = [weather_stations.loc[min_index[0], "LAT"], weather_stations.loc[min_index[1], "LAT"], weather_stations.loc[min_index[2],"LAT"]]
        dlon = [weather_stations.loc[min_index[0], "LON"], weather_stations.loc[min_index[1], "LON"], weather_stations.loc[min_index[2],"LON"]]
        if(min(dlat) < slat and slat < max(dlat) and min(dlon) < slon and slon < max(dlon)):
            for m in range(3):
                usaf = weather_stations.loc[min_index[m], "USAF"]
                #print(usaf) 
                wban = weather_stations.loc[min_index[m], "WBAN"]
                #print(wban)
                startyear = str(salm_null.loc[i , "InitialExposure"])[:4] #Extracts the year from the date string by taking the first four elements
                #print(startyear)
                endyear = str(salm_null.loc[i , "LastExposure"])[:4]  
                #print(endyear)
                filename_noaa = str(usaf) + str(wban) + ".csv" 
                for yr in range(int(startyear), int(endyear)+1): 
                    url_loop_daily = url_base + str(yr) + "/" + filename_noaa
                    #print(url_loop_daily)

                    #Identifies file on website
                    csv_current_daily = requests.get(url_loop_daily, allow_redirects = True)

                    #Sets the file name of the local imported file
                    filename_local_daily =  str(usaf) + str(wban) + "-" + str(yr) + "DAILY.csv" 
                    os.chdir('/Users/evelyn/Desktop/salm_null') #Sets path for daily data csv deposit  
                    open(filename_local_daily, "wb").write(csv_current_daily.content) #Write a csv for daily data
                    total = sum(1 for line in open(filename_local_daily))
                    if(total < 30):
                        dis[min_index[m]] = 100
                        z.append(100)
                    else:
                        z.append(0)
            if(max(z) < 50):
                break
            else: 
                continue
        else:
            dis[min_index[2]] = 100
    
    for k in range(3):
        index = min_index[k]
        salm_null.loc[i, "USAF"+str(k)] = weather_stations.loc[index, "USAF"]
        salm_null.loc[i, "WBAN"+str(k)] = weather_stations.loc[index, "WBAN"]
        salm_null.loc[i, "DIS"+str(k)] = dis[index]
    weather_stations = weather_stations.drop(columns = ["DIS"])
    

0.0000%
0.4785%
0.9569%
1.4354%
1.9139%
2.3923%
2.8708%
3.3493%
3.8278%
4.3062%
4.7847%
5.2632%
5.7416%
6.2201%
6.6986%
7.1770%
7.6555%
8.1340%
8.6124%
9.0909%
9.5694%
10.0478%
10.5263%
11.0048%
11.4833%
11.9617%
12.4402%
12.9187%
13.3971%
13.8756%
14.3541%
14.8325%
15.3110%
15.7895%
16.2679%
16.7464%
17.2249%
17.7033%
18.1818%
18.6603%
19.1388%
19.6172%
20.0957%
20.5742%
21.0526%
21.5311%
22.0096%
22.4880%
22.9665%
23.4450%
23.9234%
24.4019%
24.8804%
25.3589%
25.8373%
26.3158%
26.7943%
27.2727%
27.7512%
28.2297%
28.7081%
29.1866%
29.6651%
30.1435%
30.6220%
31.1005%
31.5789%
32.0574%
32.5359%
33.0144%
33.4928%
33.9713%
34.4498%
34.9282%
35.4067%
35.8852%
36.3636%
36.8421%
37.3206%
37.7990%
38.2775%
38.7560%
39.2344%
39.7129%
40.1914%
40.6699%
41.1483%
41.6268%
42.1053%
42.5837%
43.0622%
43.5407%
44.0191%
44.4976%
44.9761%
45.4545%
45.9330%
46.4115%
46.8900%
47.3684%
47.8469%
48.3254%
48.8038%
49.2823%
49.7608%
50.2392%
50.7177%
51.1962%
51.6746%
52.1531%
52.6316%
53.1100%
53.5885%
54.0

In [227]:
salm_null

,CDCID,GenusName,InitialExposure,LastExposure,TotalCases,Exposurestate,ExposureState,County,county,LAT,...,DIS1,USAF2,WBAN2,DIS2,W0,W1,W2,minindex0,min_index0,min_index1
0,2118,Salmonella,2009/5/7,2009/5/20,5,MO,Missouri,Cass,Cass County,38.646474,...,43.679216,999999,23908,85.125338,0.572580,0.282476,0.144943,0.0,1.0,NaN
1,2118,Salmonella,2009/5/7,2009/5/20,5,MO,Missouri,Christian,Christian County,36.969739,...,48.597971,723439,53918,57.323484,0.547283,0.245005,0.207712,NaN,2.0,NaN
2,2118,Salmonella,2009/5/7,2009/5/20,5,MO,Missouri,Jefferson,Jefferson County,38.257630,...,35.392365,724420,13997,66.903272,0.398788,0.393204,0.208008,NaN,0.0,NaN
3,2621,Salmonella,2009/9/26,2009/10/11,16,IL,Illinois,McHenry,McHenry County,42.324298,...,34.008514,726415,94854,35.857541,0.361118,0.327895,0.310987,NaN,2.0,NaN
4,2627,Salmonella,2009/5/25,2009/6/11,21,WV,West Virginia,Logan,Logan County,37.830591,...,26.222460,742079,63876,75.434824,0.425964,0.425964,0.148072,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,298168,Salmonella,2021/8/16,2021/8/19,13,SD,South Dakota,Turner,Turner County,43.308670,...,31.595543,711680,99999,39.138596,0.390958,0.336995,0.272047,NaN,2.0,NaN
205,298292,Salmonella,2020/1/20,2020/1/31,5,MO,Missouri,Franklin,Franklin County,38.413003,...,44.479718,724347,53904,49.685523,0.356951,0.339300,0.303750,NaN,2.0,NaN
206,298455,Salmonella,2021/8/3,2021/8/7,11,KY,Kentucky,Lyon,Lyon County,37.023976,...,41.368347,722781,99999,50.233914,0.383585,0.338037,0.278378,NaN,0.0,NaN
207,298611,Salmonella,2021/9/25,2021/9/25,5,CT,Connecticut,Tolland,Tolland County,41.850000,...,18.350951,725080,14740,19.081355,0.467135,0.271631,0.261234,NaN,0.0,NaN


In [179]:
for i in range(len(salm_null)):
    salm_null.loc[i, "W0"] = 1/(1/salm_null.loc[i, "DIS0"] + 1/salm_null.loc[i, "DIS1"] + 1/salm_null.loc[i, "DIS2"])/1/salm_null.loc[i, "DIS0"]
    salm_null.loc[i, "W1"] = 1/(1/salm_null.loc[i, "DIS0"] + 1/salm_null.loc[i, "DIS1"] + 1/salm_null.loc[i, "DIS2"])/1/salm_null.loc[i, "DIS1"]
    salm_null.loc[i, "W2"] = 1/(1/salm_null.loc[i, "DIS0"] + 1/salm_null.loc[i, "DIS1"] + 1/salm_null.loc[i, "DIS2"])/1/salm_null.loc[i, "DIS2"]

In [236]:
#得到每个 outbreak的最小数据的索引
salm_null["WBAN0"] = pd.to_numeric(salm_null["WBAN0"], errors='coerce').fillna('0').astype('int32')
for a in range(len(salm_null)):
    wban_unmodified = salm_null.loc[a, "WBAN0"]
    if 0 < wban_unmodified < 10000: 
        salm_null.loc[z, "WBAN0"] = str(wban_unmodified).zfill(5)
salm_null["WBAN1"] = pd.to_numeric(salm_null["WBAN1"], errors='coerce').fillna('0').astype('int32')
for b in range(len(salm_null)):
    wban_unmodified = salm_null.loc[b, "WBAN1"]
    if 0 < wban_unmodified < 10000: 
        salm_null.loc[b, "WBAN1"] = str(wban_unmodified).zfill(5)
salm_null["WBAN2"] = pd.to_numeric(salm_null["WBAN2"], errors='coerce').fillna('0').astype('int32')
for c in range(len(salm_null)):
    wban_unmodified = salm_null.loc[c, "WBAN2"]
    if 0 < wban_unmodified < 10000: 
        salm_null.loc[c, "WBAN2"] = str(wban_unmodified).zfill(5)


In [237]:
url_base = "https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/"
for i in range(len(salm_null)):
    startyear = str(salm_null.loc[i , "InitialExposure"])[:4]
    endyear = str(salm_null.loc[i , "LastExposure"])[:4]
    for yr in range(int(startyear), int(endyear)+1):
        for k in range(3):
            usaf = salm_null.loc[i, "USAF"+str(k)]
            wban = salm_null.loc[i, "WBAN"+str(k)]
            filename_noaa = str(usaf) + str(wban) + ".csv" 
            url_loop_daily = url_base + str(yr) + "/" + filename_noaa
                    
            csv_current_daily = requests.get(url_loop_daily, allow_redirects = True)
            filename_local_daily =  str(usaf) + str(wban) + "-" + str(yr) + "DAILY.csv" 
            os.chdir('/Users/evelyn/Desktop/salm_null')
            open(filename_local_daily, "wb").write(csv_current_daily.content) #Write a csv for daily data
            

In [33]:
salm_fnull = pd.read_csv('/Users/evelyn/Desktop/new-/salm_null.csv', dtype = 'str')

In [34]:
import numpy as np
path = '/Users/evelyn/Desktop/salm_null/'
store_path = '/Users/evelyn/Desktop/new-/salm_fnull/'
variable = ["TEMP", "DEWP", "SLP", "STP", "VISIB", "WDSP", "MXSPD", "GUST", "MAX", "MIN","PRCP"]
for i in range(len(salm_fnull)):
    print('{:.4%}'.format(i/len(salm_fnull)))
    if(pd.isna(salm_fnull.loc[i, "USAF2"])):
        continue
    else:
        file = []
        startyear = str(salm_fnull.loc[i , "InitialExposure"])[:4]
        endyear = str(salm_fnull.loc[i , "LastExposure"])[:4]
        w = [float(salm_fnull.loc[i, "W0"]), float(salm_fnull.loc[i, "W1"]), float(salm_fnull.loc[i, "W2"])]
        for yr in range(int(startyear), int(endyear)+1):
            for k in range(3):
                usaf = salm_fnull.loc[i, "USAF"+str(k)]
                wban = pd.to_numeric(salm_fnull.loc[i, "WBAN"+str(k)]).astype('int32')
                filename =  str(usaf) + str(wban) + "-" + str(yr) + "DAILY.csv" 
                file.append(filename)
            df0 = pd.read_csv(path + file[0])
            df1 = pd.read_csv(path + file[1])
            df2 = pd.read_csv(path + file[2])
            for j in range(len(df0)):
                date_string = df0.loc[j, "DATE"]
                if not(df1.DATE.str.contains(date_string).any() and df2.DATE.str.contains(date_string).any()):
                    df0 = df0.drop(index = [j])
            df0 = df0.reset_index(drop = True)
            for n in range(len(df0)):
                index1 = df1[df1.DATE.str.contains(df0.loc[n, "DATE"])].index.tolist()[0]
                index2 = df2[df2.DATE == df0.loc[n, "DATE"]].index.tolist()[0]
                for vr in range(len(variable)):
                    var = [float(df0.loc[n, variable[vr]]), float(df1.loc[index1, variable[vr]]), float(df2.loc[index2, variable[vr]])]
                    df0.loc[n, variable[vr]] = round(sum(np.multiply(var,w)), 1)
                
                #更改sndp
                sndp = [df0.loc[n, "SNDP"], df1.loc[index1, "SNDP"], df2.loc[index2, "SNDP"]]
                for sn in range(3):
                    if(sndp[sn] > 999):
                        sndp[sn] = 0
                df0.loc[n, "SNDP"] = round(sum(np.multiply(sndp,w)), 1)
                #对FRSHTT进行修改
                fr = [df0.loc[n, "FRSHTT"], df1.loc[index1, "FRSHTT"], df2.loc[index2, "FRSHTT"]]
                if(((fr[0]%10) + (fr[1]%10) + (fr[2]%10)) > 1):# 个位
                    df0.loc[n, "Tornado or Funnel Cloud"] = int(1)
                else:
                    df0.loc[n, "Tornado or Funnel Cloud"] = int(0)
                if(((fr[0]//10%10) + (fr[1]//10%10) + (fr[2]//10%10)) > 1):#十位
                    df0.loc[n, "Thunder"] = int(1)
                else:
                    df0.loc[n, "Thunder"] = int(0)
                if(((fr[0]//100%10) + (fr[1]//100%10) + (fr[2]//100%10)) > 1):#百位
                    df0.loc[n, "Hail"] = int(1)
                else:
                    df0.loc[n, "Hail"] = int(0)
                if(((fr[0]//1000%10) + (fr[1]//1000%10) + (fr[2]//1000%10)) > 1):#千位
                    df0.loc[n, "Snow or Ice Pellets"] = int(1)
                else:
                    df0.loc[n, "Snow or Ice Pellets"] = int(0)
                if(((fr[0]//10000 % 10) + (fr[1]//10000 % 10) + (fr[2]//10000 % 10)) > 1):
                    df0.loc[n, "Rain or Drizzle"] = int(1)
                else:
                    df0.loc[n, "Rain or Drizzle"] = int(0)
                if(((fr[0]//100000 % 10) + (fr[1]//100000 % 10) + (fr[2]//100000 % 10)) > 1):
                    df0.loc[n, "Fog"] = int(1)
                else:
                    df0.loc[n, "Fog"] = int(0)
            
            df0 = df0.drop(columns = ["FRSHTT"])      
            new_usaf = salm_fnull.loc[i, "USAF0"]
            new_wban = pd.to_numeric(salm_fnull.loc[i, "WBAN0"]).astype('int32')
            new_filename = str(new_usaf)+str(new_wban) + "-" + str(yr) + "DAILY.csv"
            df0.to_csv(store_path + new_filename)
                    

0.0000%
0.4785%
0.9569%
1.4354%
1.9139%
2.3923%
2.8708%
3.3493%
3.8278%
4.3062%
4.7847%
5.2632%
5.7416%
6.2201%
6.6986%
7.1770%
7.6555%
8.1340%
8.6124%
9.0909%
9.5694%
10.0478%
10.5263%
11.0048%
11.4833%
11.9617%
12.4402%
12.9187%
13.3971%
13.8756%
14.3541%
14.8325%
15.3110%
15.7895%
16.2679%
16.7464%
17.2249%
17.7033%
18.1818%
18.6603%
19.1388%
19.6172%
20.0957%
20.5742%
21.0526%
21.5311%
22.0096%
22.4880%
22.9665%
23.4450%
23.9234%
24.4019%
24.8804%
25.3589%
25.8373%
26.3158%
26.7943%
27.2727%
27.7512%
28.2297%
28.7081%
29.1866%
29.6651%
30.1435%
30.6220%
31.1005%
31.5789%
32.0574%
32.5359%
33.0144%
33.4928%
33.9713%
34.4498%
34.9282%
35.4067%
35.8852%
36.3636%
36.8421%
37.3206%
37.7990%
38.2775%
38.7560%
39.2344%
39.7129%
40.1914%
40.6699%
41.1483%
41.6268%
42.1053%
42.5837%
43.0622%
43.5407%
44.0191%
44.4976%
44.9761%
45.4545%
45.9330%
46.4115%
46.8900%
47.3684%
47.8469%
48.3254%
48.8038%
49.2823%
49.7608%
50.2392%
50.7177%
51.1962%
51.6746%
52.1531%
52.6316%
53.1100%
53.5885%
54.0

In [35]:
#two county intepolate
import numpy as np
path = '/Users/evelyn/Desktop/salm_null/'
store_path = '/Users/evelyn/Desktop/new-/salm_fnull/'
variable = ["TEMP", "DEWP", "SLP", "STP", "VISIB", "WDSP", "MXSPD", "GUST", "MAX", "MIN","PRCP"]
for i in range(len(salm_fnull)):
    if(pd.isna(salm_fnull.loc[i, "USAF2"]) and pd.notna(salm_fnull.loc[i, "USAF1"])):
        file = []
        startyear = str(salm_fnull.loc[i , "InitialExposure"])[:4]
        endyear = str(salm_fnull.loc[i , "LastExposure"])[:4]
        w = [float(salm_fnull.loc[i, "W0"]), float(salm_fnull.loc[i, "W1"])]
        for yr in range(int(startyear), int(endyear)+1):
            for k in range(2):
                usaf = salm_fnull.loc[i, "USAF"+str(k)]
                wban = pd.to_numeric(salm_fnull.loc[i, "WBAN"+str(k)]).astype('int32')
                filename =  str(usaf) + str(wban) + "-" + str(yr) + "DAILY.csv" 
                file.append(filename)
            df0 = pd.read_csv(path + file[0])
            df1 = pd.read_csv(path + file[1])
            for j in range(len(df0)):
                date_string = df0.loc[j, "DATE"]
                if not(df1.DATE.str.contains(date_string).any()):
                    df0 = df0.drop(index = [j])
            df0 = df0.reset_index(drop = True)
            for n in range(len(df0)):
                index1 = df1[df1.DATE.str.contains(df0.loc[n, "DATE"])].index.tolist()[0]
                for vr in range(len(variable)):
                    var = [float(df0.loc[n, variable[vr]]), float(df1.loc[index1, variable[vr]])]
                    df0.loc[n, variable[vr]] = round(sum(np.multiply(var,w)), 1)
                #更改sndp
                sndp = [df0.loc[n, "SNDP"], df1.loc[index1, "SNDP"]]
                for sn in range(2):
                    if(sndp[sn] > 999):
                        sndp[sn] = 0
                df0.loc[n, "SNDP"] = round(sum(np.multiply(sndp,w)), 1)
                #对FRSHTT进行修改
                fr = [df0.loc[n, "FRSHTT"], df1.loc[index1, "FRSHTT"]]
                if(((fr[0]%10) + (fr[1]%10)) > 1):# 个位
                    df0.loc[n, "Tornado or Funnel Cloud"] = int(1)
                else:
                    df0.loc[n, "Tornado or Funnel Cloud"] = int(0)
                if(((fr[0]//10%10) + (fr[1]//10%10)) > 1):#十位
                    df0.loc[n, "Thunder"] = int(1)
                else:
                    df0.loc[n, "Thunder"] = int(0)
                if(((fr[0]//100%10) + (fr[1]//100%10)) > 1):#百位
                    df0.loc[n, "Hail"] = int(1)
                else:
                    df0.loc[n, "Hail"] = int(0)
                if(((fr[0]//1000%10) + (fr[1]//1000%10))  > 1):#千位
                    df0.loc[n, "Snow or Ice Pellets"] = int(1)
                else:
                    df0.loc[n, "Snow or Ice Pellets"] = int(0)
                if(((fr[0]//10000 % 10) + (fr[1]//10000 % 10)) > 1):
                    df0.loc[n, "Rain or Drizzle"] = int(1)
                else:
                    df0.loc[n, "Rain or Drizzle"] = int(0)
                if(((fr[0]//100000 % 10) + (fr[1]//100000 % 10)) > 1):
                    df0.loc[n, "Fog"] = int(1)
                else:
                    df0.loc[n, "Fog"] = int(0)
            
            df0 = df0.drop(columns = ["FRSHTT"])      
            new_usaf = pd.to_numeric(salm_fnull.loc[i, "USAF0"]).astype('int32')
            new_wban = pd.to_numeric(salm_fnull.loc[i, "WBAN0"]).astype('int32')
            new_filename = str(new_usaf)+str(new_wban) + "-" + str(yr) + "DAILY.csv"
            df0.to_csv(store_path + new_filename)
    else:
                   continue
                